# Face Detection System - ProCam S.p.A

## Contesto & Obiettivi

Contesto: La ProCam S.p.A. è pronta a lanciare una nuova fotocamera digitale compatta, accessibile e pensata per i giovani appassionati di fotografia. L'obiettivo principale del prodotto è facilitare l'esperienza di scatto, in particolare per i selfie con una o più persone.

Sfida: Sei stato assunto come Data Scientist per sviluppare un sistema di rilevamento volti nelle immagini, che aiuterà i tecnici a ottimizzare automaticamente le impostazioni della fotocamera durante i selfie. Il tuo compito è realizzare una pipeline che identifichi i volti presenti nelle immagini e restituisca le coordinate dei bounding box dove i volti sono individuati. Se non ci sono volti, la pipeline restituirà una lista vuota. Si tratta di un problema di Computer Vision, più precisamente di Face Detection.

Obiettivo: Costruire un sistema di rilevamento dei volti utilizzando Scikit-learn. La pipeline deve essere in grado di:

Prendere un’immagine in ingresso.
Restituire una lista di coordinate dei bounding box dove sono presenti volti.
Restituire una lista vuota se nell’immagine non ci sono volti.

Lista articoli di documentazione:

Creating a Face Recognition system from scratch. Computer Vision with Python | A basic/intuitive approach. https://medium.com/@juanlux7/creating-a-face-recognition-system-from-scratch-83b709cd0560

FACIAL FEATURE EXTRACTION TECHNIQUES FOR FACE RECOGNITION https://thescipub.com/pdf/jcssp.2014.2360.2365.pdf

Real-Time Face Detection with HOG and SVM https://www.eeweb.com/real-time-face-detection-and-recognition-with-svm-and-hog-features/

Facial Expression Recognition Based on Facial
Components Detection and HOG Features https://cedus.it/files/3ZChi_ACV-1.pdf

OpenCV Haar Cascades https://pyimagesearch.com/2021/04/12/opencv-haar-cascades/

Performance Analysis of Face Detection by using
Viola-Jones algorithm https://www.ripublication.com/ijcir17/ijcirv13n5_05.pdf

Face Detection using Haar Cascades https://docs.opencv.org/4.x/d2/d99/tutorial_js_face_detection.html

The Viola/Jones Face Detector https://www.cs.ubc.ca/~lowe/425/slides/13-ViolaJones.pdf

ALTRO: (possibili metodologie alternative)
Object Detection with ssd, Faster RCNN, yolo https://medium.com/@javadghasemi7/object-detection-with-ssd-faster-rcnn-yolo-ce29b5c6a045

## Collegamento a kaggle

In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()  # carica il file kaggle.json
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# Librerie base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from tqdm import tqdm  # per le progress bar

# Per visualizzazione immagini in Colab
from IPython.display import Image, display
# Cerchiamo i dataset esatti
!kaggle datasets list -s "face recognition pins"
!kaggle datasets list -s "face detection images"
!kaggle datasets list -s "face emotions"
# Creiamo una directory per i dataset
!mkdir -p face_datasets

# 1. Pins Face Recognition
!kaggle datasets download hereisburak/pins-face-recognition -p face_datasets --unzip

# 2. Face Detection in Images
!kaggle datasets download dataturks/face-detection-in-images -p face_datasets --unzip

# 3. Human Face Emotions
!kaggle datasets download sanidhyak/human-face-emotions -p face_datasets --unzip

# Verificare i contenuti scaricati
!ls face_datasets



## Configurazione iniziale

In [3]:
#LIBRERIE
import numpy as np
import cv2
import joblib
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

#COSTANTI GLOBALI
TARGET_SIZE = (24, 24)
MAX_IMAGES = 5000
NEG_IMAGES = 2000
TEST_IMAGES = 3
BATCH_SIZE = 500

## Funzioni di caricamento dataset

In [4]:
def load_and_preprocess_images(directory, max_images=MAX_IMAGES):
    """Carica e preprocessa le immagini per training."""
    images = []
    files_processed = 0

    for root, _, files in os.walk(directory):
        if files_processed >= max_images:
            break

        for file in tqdm(files, desc=f"Caricamento {os.path.basename(root)}"):
            if files_processed >= max_images:
                break

            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(root, file)
                image = load_image(img_path)

                if image is not None:
                    processed = preprocess_image_hog(image)
                    if processed is not None:
                        images.append(processed)
                        images.append(cv2.flip(processed, 1))
                        files_processed += 2

    return images

def create_negative_samples(size=TARGET_SIZE, n_samples=NEG_IMAGES):
    """Genera campioni negativi per training."""
    negatives = []
    patterns = [
        lambda: np.random.randint(0, 255, size=size, dtype=np.uint8),
        lambda: np.linspace(0, 255, size[0]*size[1]).reshape(size).astype(np.uint8),
        lambda: np.fromfunction(lambda i, j: ((i+j)/2) % 255, size).astype(np.uint8)
    ]

    for _ in tqdm(range(n_samples), desc="Generazione negativi"):
        pattern = np.random.choice(patterns)()
        negatives.append(pattern)

    return negatives


## Preprocessing immagini

Questa sezione implementa due diversi approcci di preprocessing:

1. Viola-Jones Preprocessing: Equalizza, migliora il contrasto dell'immagine e mantiene le dimensioni originali

2. HOG Preprocessing: Usa equalizzazione adattiva CLAHE (Contrast Limited Adaptive Histogram Equalization) e ridimensiona l'immagine a una dimensione target fissa

In [5]:
# ===== 1. ACQUISIZIONE IMMAGINE =====
def load_image(image_path):
    """Carica e prepara l'immagine iniziale."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return image

# ===== 2. PRE-PROCESSING =====
def preprocess_image_viola_jones(image):
    """Preprocessing per Viola-Jones."""
    if image is None:
        return None
    return cv2.equalizeHist(image)

def preprocess_image_hog(image, target_size=TARGET_SIZE):
    """Preprocessing per HOG."""
    if image is None:
        return None
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    equalized = clahe.apply(image)
    return cv2.resize(equalized, target_size)

## Detection

Implementa i due metodi distinti di face detection:

Viola-Jones Detection:

Algoritmo che usa il classificatore cascade di Haar e funziona bene per volti frontali. Utilizza features rettangolari che analizzano differenze di intensità tra regioni adiacenti. E' efficace per catturare caratteristiche del volto come:

- Regione degli occhi più scura rispetto alle guance

- Ponte nasale più chiaro rispetto agli occhi

- Labbra più scure rispetto al mento



HOG Detection:

HOG è un descrittore di feature che viene combinato con AdaBoost per la classificazione includendo una fase di training e utilizza sliding window e scale pyramid per la detection

Infine la funzione non_max_suppression gestisce il raffinamento finale delle detection eliminando le detection ridondanti, selezionando il bounding box più grande come detection principale, ritornando le coordinate per il disegno del rettangolo

visualize_detections mostra quindi i risultati della detection

In [6]:
#DETECTION
def detect_faces_viola_jones(image, scale_factor=1.1, min_neighbors=8, min_size=(80, 80)):
    """Rileva volti usando Viola-Jones."""
    if image is None:
        return []

    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    )

    processed = preprocess_image_viola_jones(image)
    faces = face_cascade.detectMultiScale(
        processed,
        scaleFactor=scale_factor,
        minNeighbors=min_neighbors,
        minSize=min_size
    )

    return faces if len(faces) > 0 else []

def compute_hog_features(image):
    """Calcola le feature HOG."""
    hog = cv2.HOGDescriptor((24,24), (8,8), (4,4), (8,8), 9)
    features = hog.compute(image)
    return features.flatten()

def train_hog_detector(positive_dir, n_negative=NEG_IMAGES):
   """
   Addestra il detector HOG con ricerca parametri e validazione completa.
   """
   # Carica e prepara i dati
   print("1. Caricamento e preprocessing dati...")
   pos_images = load_and_preprocess_images(positive_dir)
   neg_images = create_negative_samples(n_samples=n_negative)

   # Estrai features
   print("\n2. Estrazione features HOG...")
   X_pos = np.array([compute_hog_features(img) for img in tqdm(pos_images, desc='HOG positivi')])
   X_neg = np.array([compute_hog_features(img) for img in tqdm(neg_images, desc='HOG negativi')])

   X = np.vstack([X_pos, X_neg])
   y = np.hstack([np.ones(len(X_pos)), np.zeros(len(X_neg))])

   print(f"\nDimensioni dataset:")
   print(f"- Esempi positivi: {len(X_pos)}")
   print(f"- Esempi negativi: {len(X_neg)}")
   print(f"- Feature per esempio: {X.shape[1]}")

   # Split del dataset
   X_train, X_test, y_train, y_test = train_test_split(
       X, y, test_size=0.2, random_state=42, stratify=y
   )

   # Parametri per grid search
   param_grid = {
       'estimator__max_depth': [2, 3],
       'n_estimators': [100, 300],
       'learning_rate': [0.01, 0.1]
   }

   # Base model
   base_model = AdaBoostClassifier(
       estimator=DecisionTreeClassifier(),
       random_state=42
   )

   # Info Grid Search
   n_combinations = len(param_grid['estimator__max_depth']) * \
                   len(param_grid['n_estimators']) * \
                   len(param_grid['learning_rate'])
   n_folds = 5
   total_fits = n_combinations * n_folds

   print(f"\n3. Inizio Grid Search:")
   print(f"- {n_combinations} combinazioni di parametri")
   print(f"- {n_folds}-fold cross validation")
   print(f"- {total_fits} fits totali\n")

   # Custom scorer per tenere traccia del progresso
   fit_params = {'current_fit': 0, 'total_fits': total_fits}

   def custom_scorer(estimator, X, y, fit_params=fit_params):
       fit_params['current_fit'] += 1
       print(f"\rFit {fit_params['current_fit']}/{fit_params['total_fits']}", end='')
       return estimator.score(X, y)

   # Grid Search
   grid_search = GridSearchCV(
       estimator=base_model,
       param_grid=param_grid,
       cv=n_folds,
       n_jobs=1,  # Set a 1 per mantenere l'ordine
       verbose=0,  # Riduciamo verbosità standard
       scoring=custom_scorer
   )

   grid_search.fit(X_train, y_train)
   print("\n") # Nuova riga dopo il completamento

   # Risultati Grid Search
   print("\n4. Risultati Grid Search:")
   print("Migliori parametri trovati:")
   print(grid_search.best_params_)
   print(f"Miglior score: {grid_search.best_score_:.3f}")

   # Modello migliore
   best_model = grid_search.best_estimator_

   # Valutazione sul test set
   print("\n5. Valutazione finale sul test set:")
   y_pred = best_model.predict(X_test)
   print("\nReport di classificazione:")
   print(classification_report(y_test, y_pred))

   # Matrice di confusione
   conf_matrix = confusion_matrix(y_test, y_pred)
   disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix,
                                display_labels=['Non-Volto', 'Volto'])
   plt.figure(figsize=(8, 6))
   disp.plot(cmap='Blues', values_format='d')
   plt.title('Matrice di Confusione')
   plt.show()

   # Learning curves
   train_sizes = np.linspace(0.1, 1.0, 5)
   train_sizes, train_scores, val_scores = learning_curve(
       best_model, X_train, y_train,
       train_sizes=train_sizes, cv=3,
       n_jobs=-1,
       scoring='f1'
   )

   plt.figure(figsize=(10, 6))
   plt.plot(train_sizes, np.mean(train_scores, axis=1), 'o-', label='Training score')
   plt.plot(train_sizes, np.mean(val_scores, axis=1), 'o-', label='Cross-validation score')
   plt.xlabel('Training examples')
   plt.ylabel('Score')
   plt.title('Learning Curves')
   plt.legend(loc='best')
   plt.grid(True)
   plt.show()

   return best_model, grid_search.best_params_, grid_search.best_score_

def detect_faces_hog(image, model, threshold=0.7):
    """Rileva volti usando HOG."""
    if image is None or model is None:
        return []

    height, width = image.shape
    min_size = 50
    scale_factor = 1.2
    stride = 8
    detections = []

    # Scale pyramid
    scales = []
    current_scale = 1.0
    while min(height * current_scale, width * current_scale) >= min_size:
        scales.append(current_scale)
        current_scale /= scale_factor

    for scale in scales:
        scaled_h = int(height * scale)
        scaled_w = int(width * scale)
        scaled_img = cv2.resize(image, (scaled_w, scaled_h))

        for y in range(0, scaled_h - min_size, stride):
            for x in range(0, scaled_w - min_size, stride):
                window = cv2.resize(scaled_img[y:y + min_size, x:x + min_size], TARGET_SIZE)
                features = compute_hog_features(window)

                prob = model.predict_proba([features])[0][1]
                if prob > threshold:
                    real_x = int(x / scale)
                    real_y = int(y / scale)
                    real_size = int(min_size / scale)
                    detections.append((real_x, real_y, real_size, real_size))

    return non_max_suppression(detections) if detections else []

#VIZ
def non_max_suppression(boxes):
    """Applica non-maximum suppression."""
    if len(boxes) == 0:
        return []

    boxes = np.array(boxes)
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,0] + boxes[:,2]
    y2 = boxes[:,1] + boxes[:,3]
    areas = (x2 - x1) * (y2 - y1)

    largest_idx = np.argmax(areas)
    return boxes[largest_idx:largest_idx+1].astype(int)

def visualize_detections(image, detections, title):
    """Visualizza i risultati della detection."""
    img_copy = cv2.cvtColor(image.copy(), cv2.COLOR_GRAY2BGR)

    for (x, y, w, h) in detections:
        cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0, 0, 255), 2)

    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f'{title}: {len(detections)} volti')
    plt.show()

In [ ]:
# Definizione del percorso del dataset
dataset_dir = os.path.join('face_datasets', '105_classes_pins_dataset')

# Verifica che il percorso esista
if not os.path.exists(dataset_dir):
    raise ValueError(f"Dataset non trovato in: {dataset_dir}")

print(f"Utilizzo dataset da: {dataset_dir}")

# Esecuzione del training con validazione
best_model, best_params, best_score = train_hog_detector(dataset_dir)

# Stampa dei risultati finali
print("\nRisultati finali:")
print("Migliori parametri:", best_params)
print("Miglior score:", best_score)

# Test su alcune immagini
test_images = []
for root, _, files in os.walk(dataset_dir):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            test_images.append(os.path.join(root, file))
            if len(test_images) >= 5:  # Test su 5 immagini
                break
    if len(test_images) >= 5:
        break

Il modello ha dimostrato ottime performance nella fase di test, raggiungendo un'accuratezza del 99% sia per il riconoscimento dei volti che dei non-volti. In particolare, la matrice di confusione rivela che su un totale di 1400 immagini di test, il sistema ha correttamente identificato 996 volti e 394 non-volti, commettendo solo 10 errori in totale (6 falsi positivi e 4 falsi negativi). Questi numeri testimoniano la robustezza e l'affidabilità del classificatore.

## Pipeline

In [ ]:
def main():
   """Pipeline principale per confronto face detection."""
   print("=== Creazione e Training Modello HOG+AdaBoost ===")

   # Crea il modello con i parametri ottimali trovati
   hog_model = AdaBoostClassifier(
       estimator=DecisionTreeClassifier(max_depth=3),
       n_estimators=300,
       learning_rate=0.1
   )

   # Carica e prepara i dati
   print("\n1. Caricamento dati...")
   positive_dir = os.path.join('face_datasets', '105_classes_pins_dataset')
   pos_images = load_and_preprocess_images(positive_dir)
   neg_images = create_negative_samples()

   # Estrai features
   print("\n2. Estrazione features...")
   X_pos = np.array([compute_hog_features(img) for img in tqdm(pos_images, desc='HOG positivi')])
   X_neg = np.array([compute_hog_features(img) for img in tqdm(neg_images, desc='HOG negativi')])

   X = np.vstack([X_pos, X_neg])
   y = np.hstack([np.ones(len(X_pos)), np.zeros(len(X_neg))])

   # Training
   print("\n3. Training modello...")
   hog_model.fit(X, y)
   print("Training completato!")

   # Test comparativo
   print("\n=== Test Comparativo ===")
   test_dir = os.path.join('face_datasets', '105_classes_pins_dataset')

   # Raccogli immagini test
   all_images = []
   for root, _, files in os.walk(test_dir):
       for file in files:
           if file.lower().endswith(('.jpg', '.jpeg', '.png')):
               all_images.append(os.path.join(root, file))
               if len(all_images) >= TEST_IMAGES:
                   break
       if len(all_images) >= TEST_IMAGES:
           break

   if all_images:
       print(f"\nTest su {TEST_IMAGES} immagini casuali...")
       selected_images = np.random.choice(
           all_images,
           size=min(TEST_IMAGES, len(all_images)),
           replace=False
       )

       for image_path in selected_images:
           print(f"\nTest su: {os.path.basename(image_path)}")
           image = load_image(image_path)

           if image is not None:
               # Test Viola-Jones
               vj_faces = detect_faces_viola_jones(image)
               visualize_detections(image, vj_faces, "Viola-Jones")

               # Test HOG
               hog_faces = detect_faces_hog(image, hog_model)
               visualize_detections(image, hog_faces, "HOG + AdaBoost")
   else:
       print("Nessuna immagine trovata per il test")

if __name__ == "__main__":
   main()

## Conclusioni

Durante lo sviluppo del progetto per ProCam S.p.A., ho seguito la consegna implementando un sistema di face detection utilizzando scikit-learn, costruendo da zero un modello basato su HOG per l'estrazione delle feature e AdaBoost per la classificazione. Il modello custom ha dimostrato buone performance, riuscendo a rilevare efficacemente i volti nelle immagini di test.
Per curiosità e confronto, abbiamo anche sperimentato con il modello Viola-Jones pre-addestrato disponibile in OpenCV. Seppur più preciso a rilevare le coordinate dei volti, il modello pre-addestrato in alcuni casi sbaglia a classificare se si tratta o meno di un volto, cosa che invece riesce bene al modello addestrato from scratch.
I risultati dei test hanno mostrato che entrambi gli approcci funzionano efficacemente per il caso d'uso dei selfie.